# Analysis of ICSE Research Track Publications by Replicability & AI Use (2023 & 2024)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
# Load Results
df_23 = pd.read_csv('results/bib/ICSE2023_papers.csv')
df_24 = pd.read_csv('results/bib/ICSE2024_papers.csv')

In [27]:
df_23.head()

,title,authors,url,abstract,artifact_available,artifact_reusable,artifact_functional
0,Log Parsing with Prompt-Based Few-Shot Learning,"Le, Van-Hoang, Zhang, Hongyu",https://doi.org/10.1109/ICSE48619.2023.00204,Logs generated by large-scale software systems...,False,False,False
1,Context-Aware Bug Reproduction for Mobile Apps,"Huang, Yuchao, Wang, Junjie, Liu, Zhe, Wang, S...",https://doi.org/10.1109/ICSE48619.2023.00196,Bug reports are vital for software maintenance...,False,False,False
2,Vulnerability Detection with Graph Simplificat...,"Wen, Xin-Cheng, Chen, Yupan, Gao, Cuiyun, Zhan...",https://doi.org/10.1109/ICSE48619.2023.00191,Prior studies have demonstrated the effectiven...,False,False,False
3,DIVER: Oracle-Guided SMT Solver Testing with U...,"Kim, Jongwook, So, Sunbeom, Oh, Hakjoo",https://doi.org/10.1109/ICSE48619.2023.00187,"We present DIVER, a novel technique for effect...",True,True,False
4,On the Robustness of Code Generation Technique...,"Mastropaolo, Antonio, Pascarella, Luca, Guglie...",https://doi.org/10.1109/ICSE48619.2023.00181,Software engineering research has always being...,False,False,False


In [31]:
def conference_overview(df, title):
    """
    Print a summary of the conference data.
    """
    print(title)
    print("------------------------")
    print(f"Total papers: {len(df)}\n")
    print(df["artifact_available"].value_counts())
    papers_with_artifacts = df[df["artifact_available"]]

    print(f"\nOf {len(papers_with_artifacts)} papers with available artifacts:")
    print(f"{papers_with_artifacts["artifact_reusable"].sum()} reusable artifacts")
    print(f"{papers_with_artifacts["artifact_functional"].sum()} functional artifacts\n\n")

In [32]:
conference_overview(df_23, "Overview for ICSE 2023")
conference_overview(df_24, "Overview for ICSE 2024")

Overview for ICSE 2023
------------------------
Total papers: 211

artifact_available
False    131
True      80
Name: count, dtype: int64

Of 80 papers with available artifacts:
51 reusable artifacts
13 functional artifacts


Overview for ICSE 2024
------------------------
Total papers: 238

artifact_available
False    159
True      79
Name: count, dtype: int64

Of 79 papers with available artifacts:
63 reusable artifacts
2 functional artifacts


